## 1.爬取文章
### coding = UTF-8
### 网址：http://www.math.pku.edu.cn/teachers/lidf/docs/textrick/index.htm
 
import urllib.request
import re
import os
 
### open the url and read

    def getHtml(url):
        page = urllib.request.urlopen(url)
        html = page.read()
        page.close()
        return html
 
### compile the regular expressions and find
### all stuff we need

    def getUrl(html):
        reg = r'(?:href|HREF)="?((?:http://)?.+?\.pdf)'
        url_re = re.compile(reg)
        url_lst = url_re.findall(html.decode('gb2312'))
        return(url_lst)

    def getFile(url):
        file_name = url.split('/')[-1]
        u = urllib.request.urlopen(url)
        f = open(file_name, 'wb')
 
    block_sz = 8192
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break
 
        f.write(buffer)
    f.close()
    print ("Sucessful to download" + " " + file_name)
 
 
    root_url = 'http://www.math.pku.edu.cn/teachers/lidf/docs/textrick/'

    raw_url = 'http://www.math.pku.edu.cn/teachers/lidf/docs/textrick/index.htm'

    html = getHtml(raw_url)
    url_lst = getUrl(html)

    os.mkdir('pdf_download')
    os.chdir(os.path.join(os.getcwd(), 'pdf_download'))

    for url in url_lst[:]:
        url = root_url + url
        getFile(url)
    
    
## 2.将PDF转换为TXT格式

    def readPDF(path, toPath):
### 以二进制形式打开pdf文件
    with open(path, "rb") as f:
### 创建一个pdf文档分析器
        parser = PDFParser(f)
### 创建pdf文档
        pdfFile = PDFDocument()
### 链接分析器与文档对象
        parser.set_document(pdfFile)
        pdfFile.set_parser(parser)
 ### 提供初始化密码
        pdfFile.initialize()
### 检测文档是否提供txt转换
    if not pdfFile.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
### 解析数据
### 数据管理
        manager = PDFResourceManager()
### 创建一个PDF设备对象
        laparams = LAParams()
        device = PDFPageAggregator(manager, laparams=laparams)
### 解释器对象
        interpreter = PDFPageInterpreter(manager, device)

### 开始循环处理，每次处理一页
        for page in pdfFile.get_pages():
            interpreter.process_page(page)
            layout = device.get_result()
            for x in layout:
                if(isinstance(x, LTTextBoxHorizontal)):
                    with open(toPath, "a",encoding='utf-8') as f:
                        str = x.get_text()
                        # print(str)
                        f.write(str+"\n")
## 3.提取TXT的关键词

    def getKeywords(filename,filename_key):
        file = open(filename, 'r',encoding='utf-8')
        file_k = open(filename_key,'a',encoding='utf-8')
        content = file.read()
        startStr = 'Keywords:'
        endStr = '\n\n'
        try:
            startIndex = content.index(startStr)
            endIndex = content[startIndex:].index(endStr)
            con_key = content[startIndex+len(startStr):startIndex+endIndex]
            file.close()
            file_k.write(con_key)
            file_k.write('\n')
            file_k.close()
        except:
           print('该文件无关键字')
            file.close()
            file_k.close()

# 由于爬虫结果没有实现，所以又做了一个爬取天气温度的例子

## 1.爬取天气数据
    import requests
    from bs4 import BeautifulSoup
    import html5lib
    import csv
    def parse_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36',
    }
    response = requests.get(url)
    text = response.content.decode('utf-8')
    # 需要用到html5lib解析器，去补全html标签
    soup = BeautifulSoup(text,'html5lib')
    conMidtab = soup.find('div',class_='conMidtab')
    tables = conMidtab.find_all('table')
    for table in tables:
        trs = table.find_all('tr')[2:]
        for index,tr in enumerate(trs):
            tds = tr.find_all('td')
            city_td = tds[0]
            if index == 0:
                city_td = tds[1]
            city = list(city_td.stripped_strings)[0]
            temp_td = tds[-2]
            temp = list(temp_td.stripped_strings)[0]
## 2.将数据存储到csv文件

            header = ['city', 'temp'] #数据列名
            f = open('文件名.csv','w',encoding='utf-8')
            csv_writer = csv.writer(f)
            list2 = [{'city':city,'temp':temp}]
            csv_writer.writerow([city,temp])
            csv_writer.writerow([city,temp])
            
    def main():
        url_list = [
            'http://www.weather.com.cn/textFC/hb.shtml',
            'http://www.weather.com.cn/textFC/db.shtml',
            'http://www.weather.com.cn/textFC/hd.shtml',
            'http://www.weather.com.cn/textFC/hz.shtml',
            'http://www.weather.com.cn/textFC/hn.shtml',
            'http://www.weather.com.cn/textFC/xb.shtml',
            'http://www.weather.com.cn/textFC/xn.shtml',
            'http://www.weather.com.cn/textFC/gat.shtml',
        ]
        for url in url_list:
            parse_page(url)
          if __name__ == '__main__':
            main()  
## 3.数据可视化

    import pandas as pd
    import matplotlib.pyplot as plt
    df = pd.read_csv('文件名.csv')
    x = df.values[:,0] #city
    y = df.values[:,1] #temp
    plt.plot(x, y, 'o-',label=u"线条")
    plt.title(u"全国各地温度折线图")
    plt.xlabel(u"city")
    plt.ylabel(u"temp")
    plt.show()
    plt.savefig("temp.png")